### Permutation mechanism

This notebook implements the mechanism of a "move", and reverse of a move

It also goes through the examples and validates that all solutions are correct.

We also try to optimize a little bit so that each step is fast.

In [2]:
import json
import copy

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [3]:
# load each puzzle and their moves
puzzle_actions = {}
puzzle_info_ndarray = pd.read_csv("./puzzles/puzzle_info.csv").to_numpy()
for puzzle_name, moves in puzzle_info_ndarray:
    puzzle_actions[puzzle_name] = json.loads(moves.replace("'", '"'))

# load all the puzzle's start and end positions
puzzles_df = pd.read_csv("./puzzles/puzzles.csv").to_numpy()

# load sample solutions
# sample_submission_df = pd.read_csv("./puzzles/sample_submission.csv").to_numpy()
sample_submission_df = pd.read_csv("./solutions/greedy_cubes.csv").to_numpy()

In [ ]:
def reverse(move):
    """returns the reversed permutation"""
    reversed_move = move[:]
    for i, j in enumerate(move):
        reversed_move[j] = i
    return reversed_move


def permute(state, move):
    """returns new state after move is performed"""
    assert len(state) == len(move)
    new_state = state[:]
    for i, j in enumerate(move):
        new_state[i] = state[j]
    return new_state


def reduce(move):
    """reduces a move to a tuple, of a list of (end, start)"""
    reduced_move = []
    for i, j in enumerate(move):
        if i != j:
            reduced_move.append((i, j))

    return reduced_move


def permute_with_reduced_move(state, reduced_move):
    new_state = state[:]
    for i, j in reduced_move:
        new_state[i] = state[j]
    return new_state


def is_valid_final_state(state, final_state, num_wild):
    assert len(state) == len(final_state)
    diff_count = 0
    for a, b in zip(state, final_state):
        if a != b:
            diff_count += 1

    return diff_count <= num_wild


reduced_actions = {pname: {} for pname in puzzle_actions.keys()}

# add reversed actions all the possible actions
for puzzle_name, actions in puzzle_actions.items():
    reversed_actions = {}
    for action_name, action in actions.items():
        reversed_actions[f"-{action_name}"] = reverse(action)
    puzzle_actions[puzzle_name].update(reversed_actions)

# get reduced actions
for puzzle_name, actions in puzzle_actions.items():
    for action_name, action in actions.items():
        reduced_actions[puzzle_name][action_name] = reduce(action)

NUM_WRONGS = 0
# play through all the sample submission moves to validate
for puzzle in puzzles_df:
    i, pname, end_state, start_state, num_wild = puzzle
    start_state = start_state.split(";")
    end_state = end_state.split(";")

    # get sample solution
    moves = sample_submission_df[i][1].split(".")

    # get action space
    actions = puzzle_actions[pname]
    actions = reduced_actions[pname]

    curr_state = start_state
    # play through the submitted actions
    for move_name in moves:
        move = actions[move_name]
        # curr_state = permute(curr_state, move)
        curr_state = permute_with_reduced_move(curr_state, move)

    if is_valid_final_state(curr_state, end_state, num_wild):
        print(f"validated puzzle {i} on {pname} with {len(moves)} moves")
    else:
        NUM_WRONGS += 1
        print(f"Puzzle {i} FAILED")

print(f'Failed on {NUM_WRONGS} puzzles')

In [19]:
moves = sample_submission_df[0][1].split(".")
moves

['r1', '-f1']